In [1]:
import torch, random
import torchvision.transforms as transforms
from torchvision.io.video import read_video
from torchvision.models.video import r3d_18, R3D_18_Weights, R2Plus1D_18_Weights,r2plus1d_18
from torch.nn import functional as F
import cv2, os
from tqdm import tqdm
import numpy as np

#weights = R3D_18_Weights.DEFAULT
weights = R2Plus1D_18_Weights.DEFAULT
model = r2plus1d_18(weights=weights)
model.eval()

VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [2]:
transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.Resize((640, 480)),  
#     transforms.RandomCrop(224),     
    transforms.ToTensor(),          
#     transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),  # Normalize
])

In [3]:
X_tr = []
labels = []

In [4]:
model1 = torch.nn.Sequential(*list(model.children())[:-1])
model1.eval()

Sequential(
  (0): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [5]:
preprocess = weights.transforms()

#### Normal

In [ ]:
path = "dataset/Normal/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

  2%|▌                                     | 118/7691 [05:18<5:02:19,  2.40s/it]

In [ ]:
check = np.array(X_tr)
print(check.shape)

#### Nudity

In [ ]:
%%time

path = "dataset/Nudity/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

In [ ]:
ctr = np.array(X_tr)
print(ctr.shape)

#### Intoxicant

In [ ]:
%%time

path = "dataset/Intoxicant/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

In [ ]:
mx = np.array(X_tr)
print(mx.shape)

#### Violance

In [ ]:
%%time

path = "dataset/Violance/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

#### Obsense

In [ ]:
%%time

path = "dataset/Obsense/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

#### Weapons

In [ ]:
%%time

path = "dataset/Weapons/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

##### Gore

In [ ]:
%%time

path = "dataset/Gore/"
listing2 = os.listdir(path)
listing2 = list(set(listing2))
for vid2 in tqdm(listing2):
    vid2 = path+vid2
    label = vid2.split("/")[1]
    cap = cv2.VideoCapture(vid2)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = 64
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    frames = []
    try:
        for frame_number in frame_numbers:    
            ret, frame = cap.read()        
            if frame is None:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame,(480,640))
            frames.append(frame)
        input_ = np.array(frames)
        video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
        video_tensor = preprocess(video_tensor)
        video_tensor = video_tensor.unsqueeze(0)
        with torch.no_grad():
            my_output = model1(video_tensor)
        pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
        feature_vector = pooled_output.flatten().cpu().numpy()
        X_tr.append(feature_vector)
        labels.append(path.split('/')[1])
    except:
        print("video error", vid2)

In [ ]:
X_tr_2= np.array(X_tr)
print(X_tr_2.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np.array(labels)
labels = to_categorical(labels)
print(le.classes_, labels.shape)

#### Split the Video and Labels

In [ ]:
print(X_tr_2.shape,labels.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=42, max_iter=256, hidden_layer_sizes=512, activation='logistic', 
                    solver='adam',learning_rate='adaptive',validation_fraction=0.15,shuffle=True,early_stopping=False,verbose=True)
clf.fit(X_tr_2, labels)

In [ ]:
import pickle

pickle.dump(clf,open("model.pkl",'wb'))
pickle.dump(le,open("labels.pkl",'wb'))

### Test

In [ ]:
import cv2
import pickle
import numpy as np

clf = pickle.load(open("model.pkl",'rb')) 
le = pickle.load(open("labels.pkl",'rb'))

In [ ]:
le.classes_

In [ ]:
video = "test/avatar.mp4"

cap = cv2.VideoCapture(video)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = 96
frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
frame_numbers.sort()
frames = []
for frame_number in frame_numbers:    
    ret, frame = cap.read()        
    if frame is None:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(480,640))
    frames.append(frame)
input_ = np.array(frames)
video_tensor = torch.stack([transform(frame) for frame in frames], dim=0)
video_tensor = preprocess(video_tensor)
video_tensor = video_tensor.unsqueeze(0)
with torch.no_grad():
    my_output = model1(video_tensor)
pooled_output = F.avg_pool3d(my_output, kernel_size=my_output.size(-2))
rgb_feature = pooled_output.flatten().cpu().numpy()
new_feature = np.expand_dims(rgb_feature,axis=0)
pred=clf.predict_proba(new_feature)[0]
print(le.inverse_transform([np.argmax(pred)])[0], pred[np.argmax(pred)], pred)            